## Visualizing Activations

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
from fastai.callbacks import HookCallback

In [2]:
path = untar_data(URLs.MNIST)

In [3]:
class StoreHook(HookCallback):
    def on_train_begin(self, **kwargs):
        super().on_train_begin(**kwargs)
        self.outputs = []
        
    def hook(self, m, i, o): 
        return o
    
    def on_batch_end(self, train, **kwargs): 
        if (train): self.outputs.append(self.hooks.stored[0])

data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.5, ds_tfms=get_transforms(), size=32, num_workers=4).normalize(imagenet_stats)
learn = cnn_learner(data, models.resnet34, metrics=[error_rate])

In [4]:
len(data.train_ds)

35000

In [5]:
learn.model[0][4][0]

BasicBlock(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [6]:
def probeModule(learn, module):
    hook = StoreHook(learn, modules=flatten_model(module))
    learn.callbacks += [ hook ]
    return hook

In [7]:
firstHook = probeModule(learn, learn.model[0][4][0])
secondHook = probeModule(learn, learn.model[0][0])

learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.722845,0.475171,0.154200,00:20


torch.Size([34944, 64, 16, 16])

In [8]:
print(torch.cat(firstHook.outputs).shape)
print(torch.cat(secondHook.outputs).shape)

torch.Size([34944, 64, 8, 8])
torch.Size([34944, 64, 16, 16])
